In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

In [ ]:
!nvidia-smi

Sat Jul 25 02:08:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    75W / 149W |    255MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 3860 (delta 1), reused 4 (delta 1), pack-reused 3853
Receiving objects: 100% (3860/3860), 148.01 MiB | 33.47 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Checking out files: 100% (3864/3864), done.


In [ ]:
from csv import writer
from imutils import paths
import imutils
import pandas as pd
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
import sklearn
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

In [ ]:
def resize_to_fit(image, width, height):
    """
    A helper function to resize an image to fit within a given size
    :param image: image to resize
    :param width: desired width in pixels
    :param height: desired height in pixels
    :return: the resized image
    """

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

In [ ]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [ ]:
LETTER_IMAGES_FOLDER0 = "with_mask"
LETTER_IMAGES_FOLDER1 = "without_mask"
row_content0 = [1,0]
row_content1 = [0,1]

In [ ]:
for image_file in paths.list_images(LETTER_IMAGES_FOLDER0):
    append_list_as_row('mask.csv', row_content0)

for image_file in paths.list_images(LETTER_IMAGES_FOLDER1):
    append_list_as_row('mask.csv', row_content1)

In [ ]:
images = []
for image_file in paths.list_images(LETTER_IMAGES_FOLDER0):
    image = cv2.imread(image_file)
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)
for image_file in paths.list_images(LETTER_IMAGES_FOLDER1):
    image = cv2.imread(image_file) 
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)

In [ ]:
images = np.array(images)
images.shape

(3846, 120, 120, 3)

In [ ]:
data = pd.read_csv('MASKF.csv')
labels = np.array(data)
labels.shape

(3846, 2)

In [ ]:
images, labels = shuffle(images, labels)


In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(images, labels, test_size=0.1)

In [ ]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=[120,120,3]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,y_train,epochs=4,callbacks=[checkpoint],validation_data=(X_test, y_test))

Train on 3461 samples, validate on 385 samples
Epoch 1/4
3461/3461 [==============================] - 21s 6ms/step - loss: 0.4302 - accuracy: 0.8214 - val_loss: 0.1760 - val_accuracy: 0.9455
Epoch 2/4
3461/3461 [==============================] - 14s 4ms/step - loss: 0.1982 - accuracy: 0.9266 - val_loss: 0.1717 - val_accuracy: 0.9325
Epoch 3/4
3461/3461 [==============================] - 14s 4ms/step - loss: 0.1599 - accuracy: 0.9413 - val_loss: 0.1244 - val_accuracy: 0.9455
Epoch 4/4
3461/3461 [==============================] - 14s 4ms/step - loss: 0.1283 - accuracy: 0.9500 - val_loss: 0.1423 - val_accuracy: 0.9429
